In [2]:
import pandas as pd
import numpy as np

In [2]:
dataset = pd.read_csv('sarcos_inv.csv')

In [3]:
dataset = dataset.to_numpy()
dataset

array([[ 1.7279000e-02, -1.3707700e-01,  2.6999000e-02, ...,
         1.0150700e+00, -1.7048688e+01,  4.4104164e+01],
       [ 1.6336000e-02, -1.4087800e-01,  2.7250000e-02, ...,
         9.1054800e-01, -1.1415526e+01,  3.7354858e+01],
       [ 1.6273000e-02, -1.4530700e-01,  2.9072000e-02, ...,
         9.2136000e-01, -5.7720580e+00,  3.0676065e+01],
       ...,
       [-5.5157700e-01, -4.4514100e-01,  7.7410000e-02, ...,
         8.3922470e+00,  1.6851494e+01,  2.2699657e+01],
       [-5.4183800e-01, -4.3612400e-01,  7.4300000e-02, ...,
         8.7151520e+00,  1.6502180e+01,  1.7126688e+01],
       [-5.3062200e-01, -4.2619700e-01,  6.9838000e-02, ...,
         8.5365940e+00,  1.5568848e+01,  6.5244520e+00]])

In [4]:
total = dataset.shape[0]
split = int(total * 0.7)

np.random.seed(0)
shuffle  = np.random.permutation(total)
dataset = dataset[shuffle]

train = dataset[:split]
test = dataset[split:]

In [5]:
x_train = train[:, :21]
y_train = train[:, 21:]

x_test = test[:, :21]
y_test = test[:, 21:]

print('Training set size:', x_train.shape[0])
print('Test set size:', x_test.shape[0])

Training set size: 31138
Test set size: 13345


In [6]:
def random_sample(dataset, rows):
    dataset = dataset[shuffle]
    return dataset[:rows]

In [7]:
#def calculate_entropy(y):
#    total = 0
#    for i in np.unique(y):
#        data = np.count_nonzero(y==i)
#        data = data / len(y)
#        data = data * np.log2(data)
#        total += data
#    total *= -1
#    return total

In [8]:
# Calculate the Gini index for a split dataset
#def gini_index(groups, classes):
#    # count all samples at split point
#    n_instances = float(sum([len(group) for group in groups]))
#    # sum weighted Gini index for each group
#    gini = 0.0
#    for group in groups:
#        size = float(len(group))
#        # avoid divide by zero
#        if size == 0:
#            continue
#        score = 0.0
#        # score the group based on the score for each class
#        for class_val in classes:
#            p = [row[-1] for row in group].count(class_val) / size
#            score += p * p
#        # weight the group score by its relative size
#        gini += (1.0 - score) * (size / n_instances)
#    return gini

In [9]:
#def find_split(x, y):
 #   """Given a dataset and its target values, this finds the optimal combination
#    of feature and split point that gives the maximum information gain."""
    
    # Need the starting entropy so we can measure improvement...
#    start_entropy = calculate_entropy(y)
    
    # Best thus far, initialised to a dud that will be replaced immediately...
#    best = {'infogain' : -np.inf}
#    
    # Loop every possible split of every dimension...
#    for i in range(x.shape[1]):
#        for split in np.unique(x[:,i]):
#            left_entropy = calculate_entropy(y[x[:, i]<= split])
#            left_indices = x[:, i] <= split
#            right_entropy = calculate_entropy(y[x[:, i]> split])
#            right_indices = x[:, i] > split
#            n1 = len(y[x[:, i]<= split])
#            nr = len(y[x[:, i]> split])
#            n = len(x)
#            infogain = start_entropy - ((n1/n)*left_entropy) - ((nr/n)*right_entropy)
#            
#            if infogain > best['infogain']:
#                best = {'feature' : i,
#                        'split' : split,
#                        'infogain' : infogain, 
#                        'left_indices' : left_indices,
#                        'right_indices' : right_indices}
#    return best

In [10]:
def find_split(x, y):
    """Given a dataset and its target values, this finds the optimal combination
    of feature and split point that gives the maximum information gain."""
    
    # Need the starting entropy so we can measure improvement...
#    start_entropy = calculate_entropy(y)
    
    # Best thus far, initialised to a dud that will be replaced immediately...
    best = {'variance' : np.inf}
    
    # Loop every possible split of every dimension...
    for i in range(x.shape[1]):
        for split in np.unique(x[:,i]):
            left_indices = x[:, i] <= split
            right_indices = x[:, i] > split
            n1 = len(y[x[:, i]<= split])
            nr = len(y[x[:, i]> split])
            n = len(x)
            left_variance = np.var(y[x[:, i]<=split])
            right_variance = np.var(y[x[:, i]>split])
            total_variance = ((n1/n)*(left_variance**2))+((nr/n)*(right_variance**2))
            
            if total_variance < best['variance']:
                best = {'feature' : i,
                        'split' : split,
                        'variance' : total_variance, 
                        'left_indices' : left_indices,
                        'right_indices' : right_indices}
    return best

In [11]:
#def build_tree(x, y, max_depth = np.inf):
#    # Check if either of the stopping conditions have been reached. If so generate a leaf node...
#    if max_depth==1 or (y==y[0]).all():
#        # Generate a leaf node...
#        classes, counts = np.unique(y, return_counts=True)
#        return {'leaf' : True, 'class' : classes[np.argmax(counts)]}
#    
#    else:
#        move = find_split(x, y)
#        left = build_tree(x[move['left_indices'],:], y[move['left_indices']], max_depth - 1)
##        right = build_tree(x[move['right_indices'],:], y[move['right_indices']], max_depth - 1)
#        
 #       return {'leaf' : False,
 #               'feature' : move['feature'],
#                'split' : move['split'],
#                'infogain' : move['infogain'],
#                'left' : left,
#                'right' : right}

In [12]:
def build_tree(x, y, max_depth = np.inf):
    # Check if either of the stopping conditions have been reached. If so generate a leaf node...
    if max_depth==1 or (y==y[0]).all():
        # Generate a leaf node...
        classes, counts = np.unique(y, return_counts=True)
        return {'leaf' : True, 'class' : classes[np.argmax(counts)]}
    
    else:
        move = find_split(x, y)
        left = build_tree(x[move['left_indices'],:], y[move['left_indices']], max_depth - 1)
        right = build_tree(x[move['right_indices'],:], y[move['right_indices']], max_depth - 1)
        
        return {'leaf' : False,
                'feature' : move['feature'],
                'split' : move['split'],
                'variance' : move['variance'],
                'left' : left,
                'right' : right}

In [13]:
def predict_one(tree, sample):
    """Does the prediction for a single data point"""
    if tree['leaf']:
        return tree['class']
    
    else:
        if sample[tree['feature']] <= tree['split']:
            return predict_one(tree['left'], sample)
        else:
            return predict_one(tree['right'], sample)

In [28]:
#build the tree with the n_trees, depth and sample size hyperparamets
def random_forest(train, n_trees = 100, depth = 7, sample_size = 2500):
    treeArr = []
    count = 0
    trees = []
    for i in range(n_trees):
        treeArr.append(random_sample(dataset, sample_size))
    for x in treeArr:
        tree=build_tree(x[:, :21], x[:,21:], depth)
        trees.append(tree)
        count+=1
    return trees
trees = random_forest(train)

In [29]:
#making predictions with random forests
def predict_random_forest(x_test, y_test):
    actual = 0
    wrong = 0
    predictions = []
    for i in range(len(x_test)):
        mean = 0
        for tree in trees:
            predict = predict_one(tree, x_test[i])
            mean += predict
        mean = mean/len(trees)
        print(mean)
        print(y_test[i])
        predictions.append(mean)
        if ((y_test[i]-1)<=mean) and (mean<=(y_test[i]+1)):
            actual +=1
        else:
            wrong +=1
        accuracy = actual/(actual+wrong) *1000
        print ('accuracy', accuracy, "%")
    print('mean accuracy', accuracy, "%")
    print('predictions: ')
    return predictions
predict_random_forest(x_test, y_test)

5.165191999999994
[16.311131]
accuracy 0.0 %
-10.194457000000027
[-17.262613]
accuracy 0.0 %
12.777053000000016
[11.009069]
accuracy 0.0 %
-43.904125000000015
[-24.602623]
accuracy 0.0 %
28.408551999999997
[15.359648]
accuracy 0.0 %
11.01001399999998
[14.27224]
accuracy 0.0 %
37.651526999999994
[28.272627]
accuracy 0.0 %
8.01963899999999
[7.204083]
accuracy 125.0 %
-18.656854999999965
[-7.923899]
accuracy 111.1111111111111 %
12.777053000000016
[35.612637]
accuracy 100.0 %
0.8155569999999988
[-10.058531]
accuracy 90.9090909090909 %
0.8155569999999988
[-4.077783]
accuracy 83.33333333333333 %
-17.670391000000045
[4.077783]
accuracy 76.92307692307693 %
1.3592610000000027
[6.388526]
accuracy 71.42857142857143 %
28.408551999999997
[29.922405]
accuracy 66.66666666666667 %
-2.0388910000000044
[-6.2526]
accuracy 62.5 %
8.01963899999999
[7.838879]
accuracy 117.6470588235294 %
42.13708899999994
[45.26339]
accuracy 111.1111111111111 %
-17.670391000000045
[-4.485561]
accuracy 105.26315789473684 %
-

11.01001399999998
[12.5052]
accuracy 69.76744186046511 %
13.864460999999974
[28.816332]
accuracy 69.4980694980695 %
0.8155569999999988
[1.957672]
accuracy 69.23076923076924 %
12.777053000000016
[20.796692]
accuracy 68.9655172413793 %
-18.656854999999965
[-8.978156]
accuracy 68.70229007633588 %
29.76781500000003
[32.758186]
accuracy 68.44106463878327 %
0.8155569999999988
[-2.44667]
accuracy 68.18181818181817 %
-17.670391000000045
[-9.107048]
accuracy 67.9245283018868 %
31.942630999999928
[37.923378]
accuracy 67.66917293233082 %
-27.864849000000035
[-27.864849]
accuracy 71.16104868913857 %
12.777053000000016
[16.175205]
accuracy 70.8955223880597 %
2.9903739999999983
[9.786678]
accuracy 70.63197026022306 %
1.3592610000000027
[-2.296212]
accuracy 70.37037037037038 %
28.408551999999997
[34.60347]
accuracy 70.11070110701107 %
28.408551999999997
[16.854836]
accuracy 69.8529411764706 %
12.777053000000016
[18.214096]
accuracy 69.5970695970696 %
1.3592610000000027
[14.544091]
accuracy 69.3430656

-8.97112200000001
[-12.097422]
accuracy 74.44168734491315 %
11.01001399999998
[13.320757]
accuracy 74.25742574257426 %
0.8155569999999988
[8.291492]
accuracy 74.07407407407408 %
28.408551999999997
[24.602623]
accuracy 73.89162561576356 %
-8.97112200000001
[2.174817]
accuracy 73.71007371007371 %
11.01001399999998
[23.65114]
accuracy 73.52941176470588 %
-10.194457000000027
[-22.427805]
accuracy 73.34963325183375 %
12.777053000000016
[18.078171]
accuracy 73.17073170731707 %
8.01963899999999
[15.173501]
accuracy 72.992700729927 %
-21.068543999999985
[14.136313]
accuracy 72.81553398058252 %
12.777053000000016
[9.650752]
accuracy 72.63922518159806 %
11.01001399999998
[0.815557]
accuracy 72.46376811594203 %
-10.194457000000027
[-23.65114]
accuracy 72.28915662650603 %
-21.068543999999985
[12.641127]
accuracy 72.11538461538461 %
8.01963899999999
[11.553718]
accuracy 71.94244604316546 %
11.01001399999998
[17.145638]
accuracy 71.77033492822966 %
8.01963899999999
[12.325152]
accuracy 71.5990453460

accuracy 71.42857142857143 %
0.8155569999999988
[-5.129924]
accuracy 71.31011608623548 %
8.01963899999999
[8.019639]
accuracy 72.8476821192053 %
12.777053000000016
[36.803047]
accuracy 72.72727272727272 %
28.408551999999997
[38.639221]
accuracy 72.60726072607261 %
73.2641599999999
[58.855995]
accuracy 72.48764415156506 %
8.01963899999999
[11.281865]
accuracy 72.36842105263158 %
28.408551999999997
[46.809082]
accuracy 72.2495894909688 %
-6.388526
[16.175205]
accuracy 72.1311475409836 %
11.01001399999998
[26.097809]
accuracy 72.01309328968904 %
31.942630999999928
[37.515602]
accuracy 71.89542483660131 %
28.408551999999997
[19.437431]
accuracy 71.77814029363785 %
-18.656854999999965
[-15.423832]
accuracy 71.6612377850163 %
12.777053000000016
[19.709284]
accuracy 71.54471544715447 %
28.408551999999997
[15.6315]
accuracy 71.42857142857143 %
28.408551999999997
[17.942244]
accuracy 71.31280388978931 %
32.214484999999975
[25.282253]
accuracy 71.19741100323624 %
-10.194457000000027
[-13.847458]

28.408551999999997
[11.961496]
accuracy 83.97932816537468 %
37.651526999999994
[43.224495]
accuracy 83.87096774193549 %
12.777053000000016
[18.893726]
accuracy 83.76288659793813 %
0.8155569999999988
[-5.946971]
accuracy 83.65508365508366 %
32.214484999999975
[38.874863]
accuracy 83.54755784061697 %
-17.670391000000045
[2.310744]
accuracy 83.4403080872914 %
42.13708899999994
[53.060059]
accuracy 83.33333333333333 %
-8.97112200000001
[-8.372181]
accuracy 84.50704225352112 %
-10.194457000000027
[-4.126742]
accuracy 84.39897698209718 %
11.01001399999998
[6.407799]
accuracy 84.29118773946361 %
12.777053000000016
[9.378901]
accuracy 84.18367346938776 %
11.01001399999998
[21.881046]
accuracy 84.07643312101911 %
31.942630999999928
[32.758186]
accuracy 85.24173027989822 %
12.777053000000016
[19.165579]
accuracy 85.13341804320203 %
12.777053000000016
[9.378901]
accuracy 85.0253807106599 %
8.01963899999999
[1.146479]
accuracy 84.91761723700887 %
-18.656854999999965
[-13.320757]
accuracy 84.810126

...

12.777053000000016
[15.223722]
accuracy 89.23467767545253 %
11.01001399999998
[12.233348]
accuracy 89.22759387155672 %
12.777053000000016
[26.913366]
accuracy 89.22051119225274 %
12.777053000000016
[43.339718]
accuracy 89.2134296372728 %
11.01001399999998
[24.874474]
accuracy 89.2063492063492 %
5.165191999999994
[20.932617]
accuracy 89.19926989921436 %
-17.670391000000045
[-28.816332]
accuracy 89.1921917156007 %
-8.97112200000001
[-12.464793]
accuracy 89.18511465524081 %
8.01963899999999
[6.93223]
accuracy 89.17803871786735 %
-17.670391000000045
[10.874087]
accuracy 89.170963903213 %
28.408551999999997
[42.680794]
accuracy 89.16389021101064 %
12.777053000000016
[25.154926]
accuracy 89.15681764099311 %
12.777053000000016
[18.893726]
accuracy 89.1497461928934 %
12.777053000000016
[19.845209]
accuracy 89.1426758664446 %
31.942630999999928
[34.389301]
accuracy 89.13560666137985 %
8.01963899999999
[26.350977]
accuracy 89.1285385774324 %
31.942630999999928
[15.903353]
accuracy 89.12147161433

accuracy 88.86283704572098 %
28.408551999999997
[26.777439]
accuracy 88.85589246639574 %
42.13708899999994
[36.564117]
accuracy 88.84894897241541 %
32.214484999999975
[32.486336]
accuracy 88.92014377246444 %
11.01001399999998
[26.908895]
accuracy 88.91319634346432 %
-6.388526
[-44.85561]
accuracy 88.90625 %
71.08934800000017
[100.449379]
accuracy 88.89930474181705 %
0.8155569999999988
[-2.854448]
accuracy 88.89236056866115 %
-8.97112200000001
[-1.433126]
accuracy 88.88541748027806 %
28.408551999999997
[41.448349]
accuracy 88.87847547641361 %
12.777053000000016
[24.058918]
accuracy 88.87153455681376 %
76.118614
[75.982681]
accuracy 88.94268311728877 %
-50.97228200000001
[-53.418953]
accuracy 88.93573826813461 %
12.777053000000016
[17.670391]
accuracy 88.92879450343536 %
-8.97112200000001
[-2.582596]
accuracy 88.92185182293699 %
12.777053000000016
[21.340397]
accuracy 88.91491022638563 %
0.8155569999999988
[-3.398493]
accuracy 88.90796971352744 %
-6.388526
[-0.407778]
accuracy 88.9010302

accuracy 88.41862254713351 %
28.408551999999997
[41.593384]
accuracy 88.4118190212373 %
28.408551999999997
[18.485949]
accuracy 88.40501654227899 %
0.8155569999999988
[0.543704]
accuracy 88.47515002308047 %
0.8155569999999988
[-2.582596]
accuracy 88.46834371874759 %
12.777053000000016
[25.839163]
accuracy 88.46153846153847 %
0.8155569999999988
[-7.932556]
accuracy 88.45473425121145 %
-23.922992999999984
[-44.17598]
accuracy 88.44793108752499 %
12.777053000000016
[21.8841]
accuracy 88.44112897023763 %
0.8155569999999988
[8.04422]
accuracy 88.43432789910796 %
-17.670391000000045
[4.757413]
accuracy 88.42752787389466 %
0.8155569999999988
[11.396152]
accuracy 88.42072889435646 %
28.408551999999997
[19.165579]
accuracy 88.41393096025217 %
0.8155569999999988
[1.902965]
accuracy 88.4071340713407 %
0.8155569999999988
[-14.856498]
accuracy 88.40033822738104 %
-6.388526
[6.524452]
accuracy 88.3935434281322 %
8.01963899999999
[9.786678]
accuracy 88.38674967335332 %
-10.194457000000027
[2.718522]


accuracy 88.59416445623341 %
5.165191999999994
[7.204083]
accuracy 88.58745074264928 %
0.8155569999999988
[-3.805931]
accuracy 88.58073804652572 %
0.8155569999999988
[-12.777053]
accuracy 88.57402636763145 %
-2.0388910000000044
[5.980748]
accuracy 88.56731570573528 %
2.9903739999999983
[2.718522]
accuracy 88.63636363636364 %
-10.194457000000027
[-10.323889]
accuracy 88.70540110597682 %
0.8155569999999988
[-16.582983]
accuracy 88.69868201787608 %
8.01963899999999
[-3.227952]
accuracy 88.69196394758768 %
-8.97112200000001
[-5.8798]
accuracy 88.68524689488035 %
13.864460999999974
[21.612247]
accuracy 88.67853085952291 %
28.408551999999997
[38.874863]
accuracy 88.67181584128427 %
1.3592610000000027
[21.748175]
accuracy 88.66510183993337 %
28.408551999999997
[40.278534]
accuracy 88.65838885523925 %
37.651526999999994
[38.059307]
accuracy 88.7273828450299 %
-17.670391000000045
[1.495187]
accuracy 88.7206661619985 %
37.651526999999994
[54.370438]
accuracy 88.71395049579895 %
-8.97112200000001

[5.165191999999994,
 -10.194457000000027,
 12.777053000000016,
 -43.904125000000015,
 28.408551999999997,
 11.01001399999998,
 37.651526999999994,
 8.01963899999999,
 -18.656854999999965,
 12.777053000000016,
 0.8155569999999988,
 0.8155569999999988,
 -17.670391000000045,
 1.3592610000000027,
 28.408551999999997,
 -2.0388910000000044,
 8.01963899999999,
 42.13708899999994,
 -17.670391000000045,
 -17.670391000000045,
 12.777053000000016,
 47.16635499999998,
 12.777053000000016,
 12.777053000000016,
 12.777053000000016,
 12.777053000000016,
 28.408551999999997,
 12.777053000000016,
 28.408551999999997,
 8.01963899999999,
 12.777053000000016,
 12.777053000000016,
 12.777053000000016,
 28.408551999999997,
 12.777053000000016,
 0.8155569999999988,
 28.408551999999997,
 28.408551999999997,
 12.777053000000016,
 5.165191999999994,
 13.864460999999974,
 0.8155569999999988,
 8.01963899999999,
 12.777053000000016,
 32.214484999999975,
 28.408551999999997,
 2.9903739999999983,
 8.01963899999999,


# TOY PROBLEM

In [3]:
#x=np.array([[3.389989,6.886696,-1.724902,-1.377955],
#            [17.828263,0.60976,-1.759618,-1.165433],
#            [-12.989385,1.987382,-1.132566,1.564803],
#            [-31.670778,0.902384,37.923378,26.777439],
#            [9.01299,13.288522,0.682191,-5.080918]])
#x.shape

(5, 4)

In [6]:
#splitting the dataset for 80% training and 20% testing 
#total2 = x.shape[0]
#split2 = int(total2 * 0.8)

#shuffling the dataset for pre processing purposes
#np.random.seed(0)
#shuffle  = np.random.permutation(total2)
#x = x[shuffle]

#train2 = x[:split]
#test2 = x[split:]

In [8]:
#preparing the x_train, y_train, x_test, y_test
#x_train2 = train2[:, :21]
#y_train2 = train2[:, 21:]

#x_test2 = test2[:, :21]
#y_test2 = test2[:, 21:]

#print('Training set size:', x_train2.shape[0])
#print('Test set size:', x_test2.shape[0])

Training set size: 4
Test set size: 1
